In [9]:
import numpy as np
import pandas as pd
from os.path import join
from datetime import datetime
from pathlib import Path

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import loggers as pl_loggers

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import BertModel
from utils.data import RelevantDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import BertModel
bert_id = "google/bert_uncased_L-2_H-128_A-2"

notebook_datetime = datetime.now()
seed_everything(1337)

Global seed set to 1337


1337

## Dataset

In [10]:
train_ds = RelevantDataset(dataset="train", 
                           target_mode="sentencetype"
                          )
valid_ds = RelevantDataset(dataset="val", 
                           target_mode="sentencetype",
                           dimensions = train_ds.dimensions
                          )

## Hyperparams

In [11]:
start_lr = 1e-4
batch_size = 16

## Loader

In [12]:
train_dl = DataLoader(train_ds,batch_size  = batch_size, shuffle=True)
valid_dl = DataLoader(train_ds, batch_size  = 64, shuffle=False)

## Model

In [14]:
model = SectorModuleV00(
    bert = BertModel.from_pretrained(bert_id).to(device),
    input_size = sum(train_ds.dimensions[0][1]), 
    output_size = train_ds.dimensions[1],
    start_lr=start_lr
)

## Logging

In [15]:
architecture_name = model.__class__.__name__
logdir = join("logs", architecture_name, notebook_datetime.strftime("%Y-%m-%dT%H-%M-%S"))
print(f"Logging to {logdir}")
Path(logdir).mkdir(parents=True, exist_ok=True)

tb_logger = pl_loggers.TensorBoardLogger(logdir, name="", version="")

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=logdir,
    verbose=True,
    save_last=True,
    save_top_k=-1, #save all
    mode="min",
    filename='-{epoch:02d}-{val_loss:.2f}'
)

Logging to logs/SectorModuleV00/2021-04-21T13-39-24


## Trainer

In [16]:
trainer = Trainer(
#     gpus=1,
    gpus=0,
#     precision=16, 
    logger=tb_logger, 
    callbacks=[checkpoint_callback],
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


## Executing run

In [17]:
trainer.fit(model, train_dl, valid_dl)


  | Name              | Type       | Params
-------------------------------------------------
0 | bert              | BertModel  | 4.4 M 
1 | linear_after_bert | Linear     | 33.0 K
2 | feed_forward      | Sequential | 713 K 
-------------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.531    Total estimated model params size (MB)
/home/loerinczy/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

tensor([ 2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  1,  1,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8, 11, 11,  4,  4,  4,  4,  4,  4, 11, 11, 11,  1,  1,
         1,  1, 11, 11, 11, 11, 11, 11, 11,  4,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  5,  5,  5,  5,  5, 11, 11])
tensor([[ 0.0577, -0.0758, -0.6495,  0.2726,  0.4726,  0.3060,  0.7423,  0.7771,
          0.4314, -0.0652, -0.1171,  0.0916],
        [ 0.1237, -0.1400, -0.7186,  0.3192,  0.4358,  0.2047,  0.6568,  0.7984,
          0.4734, -0.0870, -0.2390,  0.0898],
        [ 0.0730, -0.1629, -0.6290,  0.3139,  0.4240,  0.3041,  0.7197,  0.7357,
          0.4282, -0.0931, -0.0835,  0.0617],
        [ 0.0144, -0.0914, -0.6548,  0.2543,  0.4900,  0.2639,  0.6420,  0.7704,
          0.4974, -0.1040, -0.1421,  0.0132],
        [-0.1431, -0.2889, -0.4961,  0.3382,  0.3612,  0.1759,  0.6577,  0.7211,
          0.4560, -0.0124, -0.2096, -0.0059],
        [-0.1436, -0.3056, -0.4280,  0.2769,  0.2831,  0.2291,  0.7342,  0.6

/home/loerinczy/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/loerinczy/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...
Epoch 0, global step 1: val_loss reached 2.65534 (best 2.65534), saving model to "/home/loerinczy/Desktop/CHEERS_Challenge/CHEERS_challenge_round_1/logs/SectorModuleV00/2021-04-21T13-39-24/-epoch=00-val_loss=2.66.ckpt" as top 1


1